In [62]:
# End to End Project
# Deep Learning
# ANN classification

In [3]:
import pandas as pd
import dill

In [4]:
df = pd.read_csv('/Users/apple/Desktop/udemy/end_to_end_prjct/ANN_classification/Churn_Modelling.csv')

df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [65]:
# drop irrelavent columns
df = df.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  object 
 2   Gender           10000 non-null  object 
 3   Age              10000 non-null  int64  
 4   Tenure           10000 non-null  int64  
 5   Balance          10000 non-null  float64
 6   NumOfProducts    10000 non-null  int64  
 7   HasCrCard        10000 non-null  int64  
 8   IsActiveMember   10000 non-null  int64  
 9   EstimatedSalary  10000 non-null  float64
 10  Exited           10000 non-null  int64  
dtypes: float64(2), int64(7), object(2)
memory usage: 859.5+ KB


In [67]:
print(df['Gender'].unique())
print(df['Geography'].unique())

['Female' 'Male']
['France' 'Spain' 'Germany']


In [68]:
# train-test split
X = df.drop('Exited',axis=1)
y = df['Exited']

from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,random_state=38,stratify=y)

In [69]:
# pre-processing

from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

num_feats = X.select_dtypes(exclude='object').columns

cat_pipeline = Pipeline([
    ['ohe',OneHotEncoder(handle_unknown='ignore',sparse_output=False)]
])

num_pipeline = Pipeline([
    ['scaler',StandardScaler()]
])

transformer = ColumnTransformer([
    ['ohe',cat_pipeline,['Geography','Gender']],
    ['scaled',num_pipeline,num_feats]
])

X_train_t = transformer.fit_transform(X_train)
X_test_t = transformer.transform(X_test)

In [70]:
# Label encoding

transformer.get_feature_names_out()


array(['ohe__Geography_France', 'ohe__Geography_Germany',
       'ohe__Geography_Spain', 'ohe__Gender_Female', 'ohe__Gender_Male',
       'scaled__CreditScore', 'scaled__Age', 'scaled__Tenure',
       'scaled__Balance', 'scaled__NumOfProducts', 'scaled__HasCrCard',
       'scaled__IsActiveMember', 'scaled__EstimatedSalary'], dtype=object)

In [71]:
df_X_train_t = pd.DataFrame(X_train_t,columns=transformer.get_feature_names_out())
df_X_test_t = pd.DataFrame(X_test_t,columns=transformer.get_feature_names_out())

In [73]:
df_X_train_t.drop('ohe__Gender_Female',axis=1,inplace=True)
df_X_test_t.drop('ohe__Gender_Female',axis=1,inplace=True)

df_X_train_t.head()

,ohe__Geography_France,ohe__Geography_Germany,ohe__Geography_Spain,ohe__Gender_Male,scaled__CreditScore,scaled__Age,scaled__Tenure,scaled__Balance,scaled__NumOfProducts,scaled__HasCrCard,scaled__IsActiveMember,scaled__EstimatedSalary
0,0.0,1.0,0.0,0.0,-1.645710,-0.085977,1.041188,-0.322010,0.815780,0.645729,-1.029502,1.542594
1,0.0,1.0,0.0,1.0,0.103297,0.485342,-1.378464,1.029849,-0.906729,0.645729,-1.029502,1.555976
2,1.0,0.0,0.0,0.0,-0.766032,-1.228614,-1.378464,1.663323,-0.906729,-1.548638,-1.029502,0.400855
3,1.0,0.0,0.0,1.0,-0.114035,0.580561,-1.378464,-1.239699,-0.906729,0.645729,0.971344,-1.560350
4,0.0,1.0,0.0,1.0,-1.925137,1.342319,-1.378464,0.648766,-0.906729,-1.548638,-1.029502,-0.445268


In [78]:
# creating .pkl

with open('transformer.pkl','wb') as f:
    dill.dump(transformer,f)

In [84]:
# ANN

import tensorflow as tf
import datetime 

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Input
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard

In [90]:
# model building

model = Sequential([
    Input(shape=(X_train_t.shape[1],)),

    Dense(64,activation='relu'),
    Dense(32,activation='relu'),
    Dense(1,activation='sigmoid')
])

In [91]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [97]:
# setting up TensorBoard & EarlyStopping

log_dir = 'log/fit/' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')

board = TensorBoard(log_dir=log_dir,histogram_freq=1)

early_stop = EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)

In [98]:
# model training 

history = model.fit(
    X_train_t,y_train,
    validation_data=(X_test_t,y_test),
    epochs=100,
    callbacks=[board,early_stop]
)

Epoch 1/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8676 - loss: 0.3189 - val_accuracy: 0.8596 - val_loss: 0.3387
Epoch 2/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8685 - loss: 0.3181 - val_accuracy: 0.8576 - val_loss: 0.3403
Epoch 3/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8700 - loss: 0.3166 - val_accuracy: 0.8560 - val_loss: 0.3400
Epoch 4/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8692 - loss: 0.3159 - val_accuracy: 0.8600 - val_loss: 0.3385
Epoch 5/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8677 - loss: 0.3142 - val_accuracy: 0.8552 - val_loss: 0.3438
Epoch 6/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8719 - loss: 0.3136 - val_accuracy: 0.8588 - val_loss: 0.3401
Epoch 7/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8731 - loss: 0.3106 - val_accuracy: 0.8528 - val_loss: 0.3408
Epoch 8/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.8713 - loss: 0.3108 - val_accu

In [99]:
model.save('ann_class_model.keras')

In [ ]:
%load_ext tensorboard
%tensorboard --logdir=log/fit --port=6006